In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import config

from IPython.display import HTML

### Numerically propagate Newtons Gravity model for many particles

In [68]:
def init_r(D, n):
    r = np.random.normal(loc = D, scale = D/4, size = n)
    thetas = np.random.rand(n)*2*np.pi
    
    rs = np.zeros((n,2))
    rs[:,0] = r*np.cos(thetas)
    rs[:,1] = r*np.sin(thetas)
    return rs

def init_v(r, v):
    
    rn = np.zeros((len(r), 3))
    rn[:,:2] = r
    
    zs = np.zeros(rn.shape)
    zs[:,2] = v
    
    return np.cross(zs,rn)[:,:2]

def init_reset(D,V, n = 100):
    r = init_r(D, n) #np.random.rand(8,2)*10 #np.array([[0.,-1.], [1.,0.], [0.,1.], [-1.,0]])
    v = init_v(r, V) 
    a = np.zeros((len(r), 2))
    
    config.masses_ = np.ones(len(r))*2 #np.array([10000., 1]) #
    #config.masses_[0] = 1000
    #r[0] = [0,0]
    #v[0] = [0,0]
    
    dt = .01
    nframes = 1000
    config.r_ = np.ma.array(r, mask=False)
    config.v_ = np.ma.array(v, mask=False)
    config.a_ = np.ma.array(a, mask=False)
    config.F_ = np.zeros(len(config.r_))
    config.dir_ij_arr_ = np.ma.zeros((len(r), len(r), len(r[0])))
    return dt, nframes
    
dt, nframes = init_reset(3, 1)    

In [6]:
def dir_ij():
    """
    Update the direction matrix from each particle to every other particle
    """
    rsubst = config.r_.copy()
    for i in range(len(config.r_)):
        config.r_.mask[i] = True
        config.dir_ij_arr_[i] = config.r_ - rsubst[i]
        config.r_.mask[i] = False
    

def Forces():
    """
    Calculate the gravitational Force on each particle due to other particles
    
    returns
    -------
    Forces : Matrix with force on particle i on i:th row
    """
    dir_ij()
    
    #F = np.zeros(len(r_))
    
    norms = np.linalg.norm(config.dir_ij_arr_, axis = 2, keepdims = True) #*masses
    masses_arr = np.tile(config.masses_, len(config.r_)).reshape(config.dir_ij_arr_.shape[:2] + (1,))
    
    dirs_m = config.dir_ij_arr_/norms*masses_arr
    config.F_ = np.dot(np.diag(config.masses_), (dirs_m/norms**2).sum(axis = 1))
    return config.F_

#### From Forces calculate the everything needed :) 

In [7]:
def update_positions_and_velocities(): #r, v, dt = .01):
    
    config.a_ = np.dot(np.diag(1/config.masses_), Forces())
    config.r_ += dt*config.v_ + .5*config.a_*dt**2
    config.v_ += config.a_*dt
    
update_positions_and_velocities()    

#### Animate using pyplot

In [67]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

rnorm = 15
dt, nframes = init_reset(rnorm, .25, 200)
fig, ax = plt.subplots(figsize = (8,8))

scat = plt.scatter([], [], marker = '.', animated=True)

def data_gen():
    cnt = 0
    while cnt < nframes:
        print(cnt)
        update_positions_and_velocities()
        cnt += 1
        yield config.r_



def init():
    plt.axis('equal')
    plt.axis('off')
    ax.set_xlim(-10*rnorm,10*rnorm)
    ax.set_ylim(-10*rnorm,10*rnorm)

    return scat,

def update(data):
    update_positions_and_velocities()
    
    
    scat.set_offsets(config.r_)
    return scat,

ani = FuncAnimation(fig, update, nframes,
                    init_func=init, blit=True, 
                    interval = dt*1000)
HTML(ani.to_html5_video())
#plt.clf()

/usr/local/lib/python3.5/dist-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
